In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [14]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # 3 Convolutional Layers with padding to maintain image dimensions
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 32 filters
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 64 filters
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 128 filters

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)  # CIFAR-10 image is 32x32, after pooling it reduces to 4x4
        self.fc2 = nn.Linear(512, 10)  # CIFAR-10 has 10 classes

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # Apply convolutions + ReLU activations + Pooling
        x = self.pool(torch.relu(self.conv1(x)))  # Conv1 -> Pool1
        x = self.pool(torch.relu(self.conv2(x)))  # Conv2 -> Pool2
        x = self.pool(torch.relu(self.conv3(x)))  # Conv3 -> Pool3

        # Flatten the tensor for the fully connected layer
        x = x.view(-1, 128 * 4 * 4)  # Flatten the output to fit into the fully connected layers

        x = torch.relu(self.fc1(x))  # Fully connected layer 1
        x = self.fc2(x)  # Output layer for classification (10 classes)

        return x  # Ensure the model returns the output (this was missing previously)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# Data transformations: Random horizontal flip, random crop, and normalization
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616]),
])

# Download and load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [17]:
# Initialize the CNN model
model = SimpleCNN().to(device)

# Loss function: Cross-Entropy Loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# Optimizer: Adam optimizer with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Print the model architecture (optional)
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [24]:
# Number of epochs for training
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Iterate through the training dataset
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update running loss and accuracy
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print the results for the current epoch
    train_loss = running_loss / len(trainloader)
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} - Train Acc: {train_acc:.2f}%")

Epoch 1/10 - Loss: 0.3895 - Train Acc: 86.26%
Epoch 2/10 - Loss: 0.3797 - Train Acc: 86.71%
Epoch 3/10 - Loss: 0.3788 - Train Acc: 86.78%
Epoch 4/10 - Loss: 0.3674 - Train Acc: 87.06%
Epoch 5/10 - Loss: 0.3654 - Train Acc: 87.27%
Epoch 6/10 - Loss: 0.3633 - Train Acc: 87.29%
Epoch 7/10 - Loss: 0.3613 - Train Acc: 87.44%
Epoch 8/10 - Loss: 0.3571 - Train Acc: 87.53%
Epoch 9/10 - Loss: 0.3504 - Train Acc: 87.70%
Epoch 10/10 - Loss: 0.3477 - Train Acc: 87.84%


In [25]:
# Test the model after training
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Turn off gradients during evaluation
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 80.94%
